### Customer Lifetime Value (CLV) model using Google Analytics Data and BQML.
##### CLV with different components including churn, propensity to buy, amount of purchase. 
##### Updates planned for improvement: Selection of high propensity to buy customers as input for value models.

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery
# GA4 CLV Model

# This code creates training data for CLV models from GA4 sample data in bigquery public dataset.
# Data is available from "20201101" to "20210131" (3 full months).
# Reference date: "20201231", prediction window is one month (Jan 2021).
# History window: 2 months (all aggregations are made based on last 2 months)
# This model is developed for understanding new customers or making decisions based on 
# recent behaviour of existing customers.

CREATE OR REPLACE TABLE   `clv.clv_ga4_train` AS (
 WITH
   visitors AS (
   SELECT
     user_pseudo_id,
     MIN(CASE WHEN ecommerce.purchase_revenue > 0 THEN event_date END ) AS first_purchase_date,
     MAX(CASE WHEN ecommerce.purchase_revenue > 0 THEN event_date END ) AS last_purchase_date,
     MIN(CASE WHEN event_name = 'page_view' THEN event_date END ) AS first_visit_date,
     SUM(ecommerce.purchase_revenue) AS purchase_total,
     SUM(CASE WHEN ecommerce.purchase_revenue > 0 THEN 1 ELSE 0 END ) AS purchase_count,
     SUM(ecommerce.unique_items) AS items_total,
     SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
     SUM(IF(event_name = 'view_item', 1, 0)) AS cnt_view_item,
     SUM(IF(event_name = 'first_visit', 1, 0)) AS cnt_first_visit,
     SUM(IF(event_name = 'session_start', 1, 0)) AS cnt_session_start,
     SUM(IF(event_name = 'begin_checkout', 1, 0)) AS cnt_begin_checkout,
     SUM(IF(event_name = 'page_view', 1, 0)) AS cnt_page_view,
     SUM(IF(event_name = 'scroll', 1, 0)) AS cnt_scroll,
     SUM(IF(event_name = 'select_promotion', 1, 0)) AS cnt_select_promotion,
     SUM(IF(event_name = 'purchase', 1, 0)) AS cnt_purchase,
     SUM(IF(event_name = 'view_promotion', 1, 0)) AS cnt_view_promotion,
     SUM(IF(event_name = 'select_item', 1, 0)) AS cnt_select_item,
     SUM(IF(event_name = 'add_shipping_info', 1, 0)) AS cnt_add_shipping_info,
     SUM(IF(event_name = 'add_to_cart', 1, 0)) AS cnt_add_to_cart,
     SUM(IF(event_name = 'view_search_results', 1, 0)) AS cnt_view_search_results,
     SUM(IF(event_name = 'add_payment_info', 1, 0)) AS cnt_add_payment_info,
     count(distinct geo.region) as cnt_region,
   FROM
     `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
   WHERE _TABLE_SUFFIX BETWEEN "20201101" AND "20201231"
   GROUP BY
     user_pseudo_id ),
 
   visitor_city AS (
   SELECT
     user_pseudo_id,
     geo_country,
     geo_region,
     visits as visits_from_region
   FROM (
     SELECT *,
       ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY visits DESC) AS row_num
     FROM (
       SELECT
         user_pseudo_id,
         geo.country as geo_country,
         geo.region as geo_region,
         COUNT(*) AS visits
       FROM
         `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
          WHERE _TABLE_SUFFIX BETWEEN "20201101" AND "20201231"
          group by 1,2,3))
       WHERE row_num = 1),
 
   visit_days AS (
       SELECT *,
       (ifnull(pages_viewed_on_day_1,0)+ ifnull(pages_viewed_on_day_7,0)) as pages_viewed_on_weekend, # Sunday:1, Saturday: 7
       (ifnull(pages_viewed_on_day_2,0)+ ifnull(pages_viewed_on_day_3,0)+ ifnull(pages_viewed_on_day_4,0)+ ifnull(pages_viewed_on_day_5,0)+ ifnull(pages_viewed_on_day_6,0)) as pages_viewed_on_weekday,
       (ifnull(purchased_on_day_1,0)+ ifnull(purchased_on_day_7,0)) as purchased_on_weekend,
       (ifnull(purchased_on_day_2,0)+ ifnull(purchased_on_day_3,0)+ ifnull(purchased_on_day_4,0)+ ifnull(purchased_on_day_5,0)+ ifnull(purchased_on_day_6,0)) as purchased_on_weekday,
       FROM
       (
           SELECT
           user_pseudo_id,
           EXTRACT(DAYOFWEEK  FROM  PARSE_DATE('%Y%m%d', event_date)) AS day,
           SUM(CASE WHEN event_name = 'page_view' THEN 1  ELSE 0 END) AS pages_viewed_on_day,
           SUM(CASE WHEN event_name = 'purchase' THEN 1  ELSE 0 END) AS purchased_on_day
           FROM
           `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
           WHERE _TABLE_SUFFIX BETWEEN "20201101" AND "20201231"
           GROUP BY
           user_pseudo_id,
           day
           )
       PIVOT(SUM(pages_viewed_on_day) pages_viewed_on_day, SUM(purchased_on_day) purchased_on_day FOR day IN (1,2,3,4,5,6,7))),
 
   engagement AS (
   SELECT
     user_pseudo_id,
     safe_divide(SUM(session_engaged),COUNT(DISTINCT session_id)) AS engagement_rate,
     ROUND(SUM(engagement_time_msec)/1000) AS engagement_time_seconds,
     safe_divide(SUM(DISTINCT  CASE  WHEN session_engaged = 0 THEN 1 ELSE 0 END),COUNT(DISTINCT session_id)) AS bounce_rate,
     COUNT(DISTINCT session_id) AS total_sessions
   FROM (
     SELECT
       user_pseudo_id,
       (
       SELECT value.int_value       FROM    UNNEST(event_params)  WHERE    key = 'ga_session_id') AS session_id,
       MAX((SELECT  value.int_value FROM    UNNEST(event_params)  WHERE    key = 'session_engaged')) AS session_engaged,
       MAX((SELECT  value.int_value FROM    UNNEST(event_params)  WHERE    key = 'engagement_time_msec')) AS engagement_time_msec
     FROM
       `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
     WHERE _TABLE_SUFFIX BETWEEN "20201101" AND "20201231"
     GROUP BY
       user_pseudo_id,
       session_id)
   GROUP BY
     user_pseudo_id ),
 
   users_sessions AS (
   SELECT *
   FROM (
     SELECT
       user_pseudo_id,
       SUM(CASE  WHEN event_name = 'page_view' THEN 1 ELSE 0 END) / COUNT(DISTINCT user_pseudo_id) AS avg_session_depth,
       MAX(CASE  WHEN device.category = 'mobile' THEN 1 ELSE 0 END ) AS mobile,
       MAX(CASE  WHEN device.web_info.browser = 'Chrome' THEN 1  ELSE   0 END ) AS chrome,
       MAX(CASE  WHEN device.web_info.browser = 'Safari' THEN 1  ELSE   0 END ) AS safari,
       MAX(CASE  WHEN device.web_info.browser <> 'Chrome' AND device.web_info.browser NOT LIKE '%Safari%' THEN 1 ELSE 0 END) AS browser_other,
       SUM(CASE  WHEN traffic_source.medium = '(none)' THEN 1    ELSE   0 END ) AS visits_traffic_source_none,
       SUM(CASE  WHEN traffic_source.medium = 'organic' THEN 1   ELSE   0 END ) AS visits_traffic_source_organic,
       SUM(CASE  WHEN traffic_source.medium = 'cpc' THEN 1       ELSE   0 END ) AS visits_traffic_source_cpc, SUM(CASE WHEN traffic_source.medium = 'cpm' THEN 1          ELSE          0        END          ) AS visits_traffic_source_cpm,
       SUM(CASE  WHEN traffic_source.medium = 'affiliate' THEN 1 ELSE   0 END ) AS visits_traffic_source_affiliate,
       SUM(CASE  WHEN traffic_source.medium = 'referral' THEN 1  ELSE   0 END ) AS visits_traffic_source_referral,
       COUNT(DISTINCT EXTRACT(DAYOFWEEK FROM  PARSE_DATE('%Y%m%d', event_date))) AS num_diff_days_visited,       
     FROM
       `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
   WHERE
       _TABLE_SUFFIX BETWEEN "20201101" AND "20201231"
     GROUP BY
       user_pseudo_id)),
 
  clv_target AS (
   SELECT
     user_pseudo_id,
     SUM(ecommerce.purchase_revenue) AS label_purchase_sum,
     SUM(CASE WHEN ecommerce.purchase_revenue > 0 THEN 1 ELSE 0 END ) AS label_purchase_cnt,
     MAX(CASE WHEN ecommerce.purchase_revenue > 0 THEN 1 ELSE 0 END ) AS label_purchase_flg,
     SUM(IF(event_name = 'select_item', 1, 0)) AS label_select_item_cnt,
     SUM(IF(event_name = 'add_shipping_info', 1, 0)) AS label_add_shipping_info_cnt,
     SUM(IF(event_name = 'add_to_cart', 1, 0)) AS label_add_to_cart_cnt,
   FROM
     `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
   WHERE
     _TABLE_SUFFIX > "20201231"
   GROUP BY
     user_pseudo_id )
 
   SELECT
       a.*,
       if(first_purchase_date is NULL, 0, 1) as ever_purchased,
       case when first_visit_date is null then 0
       else DATE_DIFF(PARSE_DATE('%Y%m%d',first_visit_date), PARSE_DATE('%Y%m%d','20201231'), DAY) end as days_since_first_visit,
       b.* EXCEPT(user_pseudo_id),
       c.* EXCEPT(user_pseudo_id),
       d.* EXCEPT(user_pseudo_id),
       e.* EXCEPT(user_pseudo_id),
       f.* EXCEPT(user_pseudo_id),
       #CASE WHEN RAND() < 0.10 THEN 1 ELSE 0 END AS hold_out
   FROM visitors a
     LEFT JOIN visitor_city b
     ON    a.user_pseudo_id = b.user_pseudo_id
     LEFT JOIN  visit_days c
     ON    a.user_pseudo_id = c.user_pseudo_id
     LEFT JOIN  engagement d
     ON    a.user_pseudo_id = d.user_pseudo_id
     LEFT JOIN  users_sessions e
     ON    a.user_pseudo_id = e.user_pseudo_id
     LEFT JOIN  clv_target f
     ON    a.user_pseudo_id = f.user_pseudo_id
);
 
# Preliminary analysis of target labels.
select
label_purchase_flg,
ever_purchased,
count(user_pseudo_id) as cnt,
round(avg(cnt_view_item),2) as view_cnt,
round(avg(cnt_add_to_cart),2) as add_to_cart_avg, 
round(avg(label_purchase_cnt),2) as purchase_cnt,
round(avg(label_purchase_sum),2) as purchase_sum
from  `clv.clv_ga4_train`
group by label_purchase_flg, ever_purchased
order by label_purchase_flg;
 
# Select active customers, define churn and remove unnecessary features in the training table.
CREATE or REPLACE TABLE `clv.clv_ga4_train_active` AS
select * EXCEPT (label_purchase_flg, label_purchase_cnt, label_purchase_sum,
                 label_select_item_cnt, label_add_shipping_info_cnt, label_add_to_cart_cnt,
                 first_purchase_date, last_purchase_date, first_visit_date, visits_from_region,
                 geo_country, geo_region, visits_traffic_source_none, browser_other, chrome, safari, 
                 bounce_rate, engagement_rate),
if(label_purchase_flg is null, 0, label_purchase_flg) as label_purchase_flg,
if(label_purchase_cnt is null, 0, label_purchase_cnt) as label_purchase_cnt,
if(label_purchase_sum is null, 0, label_purchase_sum) as label_purchase_sum,
if((label_purchase_flg < 1 and label_select_item_cnt < 1 and label_add_shipping_info_cnt <1 and
    label_add_to_cart_cnt < 1) , 1, 0) as label_churn_flg
from `clv.clv_ga4_train`
where cnt_select_item > 0 or cnt_add_to_cart > 0 or cnt_add_shipping_info > 0 or ever_purchased > 0 ;

# Preliminary analysis of active customers.
select
label_purchase_flg,
label_churn_flg,
ever_purchased,
count(user_pseudo_id) as cnt,
round(avg(cnt_view_item),2) as view_cnt,
round(avg(cnt_add_to_cart),2) as add_to_cart_avg, 
round(avg(label_purchase_cnt),2) as purchase_cnt,
round(avg(label_purchase_sum),2) as purchase_sum
from  `clv.clv_ga4_train_active`
group by label_purchase_flg, ever_purchased, label_churn_flg
order by label_purchase_flg;

# Build an unsupervised model with kmeans using a subset of the variables.
CREATE OR REPLACE MODEL
 clv.clv_kmeans_narrow OPTIONS(model_type='kmeans', kmeans_init_method = "kmeans++" #, num_clusters=7, standardize_features=true
 ) AS
   SELECT purchase_total, purchase_count, num_diff_days_visited, items_total, total_sessions
   FROM
  `clv.clv_ga4_train_active`;

# Build an unsupervised model with kmeans with a wide scope of variables.
CREATE OR REPLACE MODEL
 clv.clv_kmeans_wide OPTIONS(model_type='kmeans', kmeans_init_method = "kmeans++" #, num_clusters=7, standardize_features=true
 ) AS
   SELECT * EXCEPT (user_pseudo_id, label_purchase_cnt, label_purchase_sum, label_purchase_flg)
   FROM
  `clv.clv_ga4_train_active`;
 
# Train a regression model.
create or replace model `clv.clv_reg_propensity`
  options(MODEL_TYPE='LOGISTIC_REG',INPUT_LABEL_COLS = ["label_purchase_flg"], 
          L1_REG=0.005, enable_global_explain=TRUE)
as select
   * EXCEPT(user_pseudo_id, label_purchase_cnt, label_purchase_sum)
from  `clv.clv_ga4_train_active`;
 
# Train a boosted tree model.
create or replace model `clv.clv_btree_propensity`
  options(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',INPUT_LABEL_COLS = ["label_purchase_flg"], 
          SUBSAMPLE=0.8, enable_global_explain=TRUE)
AS
select 
  * EXCEPT(user_pseudo_id, label_purchase_cnt, label_purchase_sum)
from  `clv.clv_ga4_train_active`;

# Train a boosted tree model for value prediction.
create or replace model `clv.clv_btree_value`
options(MODEL_TYPE='BOOSTED_TREE_REGRESSOR',INPUT_LABEL_COLS = ["label_purchase_sum"], SUBSAMPLE=0.8, enable_global_explain=TRUE)
AS
select
   * EXCEPT(user_pseudo_id, label_purchase_cnt, label_purchase_flg)
from  `clv.clv_ga4_train_active`;
 
select * from ML.WEIGHTS(MODEL `clv.clv_reg_propensity`, struct(true as standardize));
select * from ML.GLOBAL_EXPLAIN(MODEL `clv.clv_reg_propensity`);
select * from ML.GLOBAL_EXPLAIN(MODEL `clv.clv_btree_propensity`);
select * from ML.GLOBAL_EXPLAIN(MODEL `clv.clv_btree_value`);

# end of file.